ANALYSIS STUDIO GHIBLI FILMOGRAPHY

In [ ]:
import pyodbc
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np

In [ ]:
sg = pd.read_excel('studio_ghiblidb.xlsx')
cast = pd.read_excel('cast sgdb.xlsx')

The following DataBase containes data about Studio Ghibli, one of the worlds' famous and successful Japanese animation studio . 
Here you'll find information about the studio filmography; deatails about the movies, casts, leading crew and expensses. 

What is in the df?

In [ ]:
sg.info()

In [ ]:
sg.head()

In [ ]:
sg.describe()

The float format had been imported incorrectly, so I reformatted it.

In [ ]:
pd.set_option('display.float_format', '{:.0f}'.format)

For better analysing i'll add a new column 'Year' based on the column, 'JapanReleaseDate' - the first release of each film


In [ ]:
sg = sg.assign(Year = sg['JapanReleaseDate'].dt.year)

In [ ]:
sg.info()

In the following queries let's understand  the studio financial curve through time:  

In which years were Studio Ghibli films released, and were multiple films released in any of those years?

In [ ]:
sg.groupby(['Year'])[['Movieid']].count()

In [ ]:
movie_count_by_year = sg.groupby('Year')[['Movieid']].count().reset_index()
movie_count_by_year.columns = ['Year', 'MovieCount']

plt.figure(figsize=(10, 4))
sb.scatterplot(data=movie_count_by_year, x='Year', y='MovieCount', s=100, color='#ffa500')
plt.title('Number of Movies by Year')
plt.xlabel('Year')
plt.ylabel('Number of Movies')
years = movie_count_by_year['Year'].unique()
plt.xticks(years, rotation=45)
y_max = movie_count_by_year['MovieCount'].max()
plt.yticks(range(0, y_max + 2, 1)) 
plt.tight_layout()
plt.show()

What was the sum of revenue vers budget each year?

In [ ]:
sg.groupby(['Year'])[['Revenue', 'Budget']].sum()


In [ ]:
sum_budget_revenue_year = sg.groupby('Movieid')[['Budget', 'Revenue']].sum().reset_index()
sum_budget_revenue_year = pd.merge(sum_budget_revenue_year, sg[['Movieid', 'MovieName']].drop_duplicates(), on='Movieid')
sum_budget_revenue_year = sum_budget_revenue_year.sort_values(by='Movieid')

x_range = sum_budget_revenue_year['Movieid']

plt.figure(figsize=(14, 8))
sb.barplot(data=sum_budget_revenue_year, x='MovieName', y='Revenue', label='Sum Revenue', color='#9c6be1')
sb.barplot(data=sum_budget_revenue_year, x='MovieName', y='Budget', label='Sum Budget', alpha=0.5, color='orange')
plt.legend()
plt.title('Sum Budget and Revenue by Movie')
plt.xlabel('Movie Name')
plt.ylabel('Sum Budget and Revenue')
plt.xticks(rotation=60, ha='right') 
plt.show()

Let's divide the budget and revnue to two separate grapghs and show it next to revenue per film

In [ ]:
revenue_by_year = sg.groupby(['Year'])[['Revenue']].sum()
budget_by_year = sg.groupby(['Year'])[['Budget']].sum()
revenue_by_movie = sg.groupby('Movieid')[['Revenue']].sum()

plt.figure(figsize=(18, 6))
plt.suptitle('Yearly & Movie-Based Expenses', fontsize=20)

ax1 = plt.subplot(1, 3, 1)
ax1.bar(revenue_by_year.index, revenue_by_year['Revenue'], color='y')
ax1.set_title('Revenue by Year')
ax1.set_xlabel('Year')
ax1.set_ylabel('Revenue')

ax2 = plt.subplot(1, 3, 2, sharey=ax1)
ax2.bar(budget_by_year.index, budget_by_year['Budget'], color='c')
ax2.set_title('Budget by Year')
ax2.set_xlabel('Year')

ax3 = plt.subplot(1, 3, 3, sharey=ax1)
ax3.bar(revenue_by_movie.index, revenue_by_movie['Revenue'], color='b')
ax3.set_title('Revenue by Movie')
ax3.set_xlabel('Movie ID')
movie_ids = revenue_by_movie.index
ax3.set_xticks(movie_ids)
ax3.set_xticklabels(movie_ids, rotation=45)


plt.tight_layout()
plt.show()


 This DataBase datetime type is based on two source,s the origin relase date in Japan and in the major worldwide boxoffice, the United States:

Which movie was the most profitable and when it was released in Japan and the USA?

In [ ]:
sg.loc[sg['Revenue']==sg['Revenue'].max(),['MovieName','Revenue','US_Release', 'JapanReleaseDate']]

In [ ]:
top_10_revenue = sg.nlargest(10, 'Revenue')
sb.barplot(data=top_10_revenue, x='Revenue', y='MovieName', color='#fabe77')
plt.title('Top 10 Movies by Revenue')
plt.show()

How many unique genres are in the studio filmography? 

In [ ]:
sg['Genre3'].unique()

In [ ]:
sg.loc[sg['Genre3']=='adventure',['MovieName','Genre3']]

In [ ]:
filtered_df = sg[['MovieName', 'Genre3']]

plt.figure(figsize=(8, 6))
sb.scatterplot(data=filtered_df, x='Genre3', y='MovieName', hue='Genre3', palette='tab10', s=150)
plt.title('Movies by distinct genre')
plt.xlabel('Genre')
plt.ylabel('Movie Name')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

I want to add the human factor and analyse the stodio finance by creators and producors

In [ ]:
How many directors worked on how many films 

In [ ]:
sg.groupby(['Director'])[['Movieid']].count()

How many movies were made in the 21st century?

In [ ]:
 sg.loc[sg['Year'].between(2000, 2023), ['MovieName', 'Year', 'Budget', 'Revenue', 'Director']]

What was the first movie released by Studio Ghibli?

In [ ]:
sg.loc[sg['JapanReleaseDate'] == sg['JapanReleaseDate'].min(), ['MovieName', 'JapanReleaseDate', 'Director']]

What was the budget for this, and what revenue did it generate?

In [ ]:
sg.loc[sg['MovieName']=='Nausicaä of the Valley of the Wind',['Budget','Revenue']]

How many movies were released during the studio's first decade of production?

In [ ]:
sg.loc[sg['Year'].between(1980,1989), ['MovieName', 'Year', 'Budget', 'Revenue', 'Director']]

How many movies were made in each decade since the studio started creating and what was the precentage of each movie from the decade's sum of revenues

In [ ]:
# Generate a unique color for each Movie ID
unique_movie_ids = sg['Movieid'].unique()
colors = sb.color_palette("husl", len(unique_movie_ids))

decades = [
    (1980, 1989),
    (1990, 1999),
    (2000, 2009),  
    (2010, 2024)
]

# Create a mapping from Movieid to color
movie_id_to_color = {movie_id: colors[i] for i, movie_id in enumerate(unique_movie_ids)}

# Define a function to format the revenue values
def format_revenue(val):
    if val >= 1e9:
        return f'{val / 1e9:.1f}B'
    elif val >= 1e6:
        return f'{val / 1e6:.1f}M'
    elif val >= 1e3:
        return f'{val / 1e3:.1f}K'
    else:
        return f'{val:.0f}'

fig, axes = plt.subplots(2, 2, figsize=(20, 20))  
for i, (start, end) in enumerate(decades):
    ax = axes[i // 2, i % 2]  # Adjust indexing for a 2x2 grid
    df_range = sg[(sg['Year'] >= start) & (sg['Year'] <= end)]
    df_grouped = df_range.groupby('Movieid')['Revenue'].sum().reset_index()
    
    labels = df_grouped['Movieid'].apply(lambda x: sg[sg['Movieid'] == x]['MovieName'].values[0])
    sizes = df_grouped['Revenue']
    colors = [movie_id_to_color[mid] for mid in df_grouped['Movieid']]
    explode = [0.1 if size == max(sizes) else 0 for size in sizes]
    wedges, texts, autotexts = ax.pie(sizes, labels=labels, colors=colors, explode=explode, 
                                      autopct='%1.1f%%', 
                                      startangle=140, textprops={'fontsize': 14}) 
    ax.set_title(f'{start}-{end}', fontsize=26)  
    
    for text in texts + autotexts:
        text.set_fontsize(14)  

plt.subplots_adjust(hspace=0.4, wspace=0.00) 
plt.tight_layout()
plt.show()

Which director made the most revenue for the studio?

In [ ]:
sg = sg.sort_values(by='Year', ascending=True)

plt.figure(figsize=(14, 8))
sb.barplot(data=sg, x="Year", y="Revenue", hue='Director', width=0.9)
plt.xticks(rotation=45)
plt.legend(title='Director', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='large')
plt.title('Revenue by Year and Director')
plt.xlabel('Year')
plt.ylabel('Revenue')
plt.show()

Who was the producer that invested the highest budget?

In [ ]:
plt.figure(figsize=(14, 8))
sb.barplot(data=sg, x='Year', y='Budget', hue='Producer', width=1)
plt.legend(title='Producer', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='large')
plt.xticks(rotation=45)
plt.title('Budget by Year and Producer')
plt.show()

How many characters are there in each movie?

In [ ]:
ch = pd.merge(cast, sg[['Movieid','MovieName']]
              , left_on='MovieId',
             right_on = 'Movieid',
             how='inner')

In [ ]:
ch.groupby('MovieName')[['CharacterName']].count()

In [ ]:
character_count = ch.groupby(['MovieName', 'Gender']).size().reset_index(name='CharacterCount')

sb.catplot(x='CharacterCount', y='MovieName', hue='Gender', data=character_count, kind='swarm', palette=('#f887ff','#832b28','c'),s=30)
plt.title('Number of Characters in a Movie')
plt.xlabel('Character Count')
plt.ylabel('Movie Name')
maxcount = character_count['CharacterCount'].max()
plt.xticks(range(1,maxcount+1), rotation=45)
plt.show()

In [ ]:
How many of the participating characters are females

In [ ]:
ch[ch['Gender'] == 'F'].groupby('MovieName')[['CharacterName']].count()

Show one movie full cast - characters and dubbing actors from Japan and the USA

In [ ]:
ch[ch['MovieName']=='Ponyo']

How many actors have done voice dubbing in more than one movie,
who are they from both the Japanese and American casts, and in which movies did they appear?

In [ ]:
japan_actor_movie_count = ch.groupby('japanCastMember').size().reset_index(name='Count')
actors_more_than_once = japan_actor_movie_count[japan_actor_movie_count['Count'] > 1]

actors_more_than_once_movies = ch[ch['japanCastMember'].isin(actors_more_than_once['japanCastMember'])]
actors_movie_list = actors_more_than_once_movies.groupby('japanCastMember')['MovieName'].apply(lambda x: ', '.join(x)).reset_index()
actors_more_than_once = actors_more_than_once.merge(actors_movie_list, on='japanCastMember')

fig, ax = plt.subplots(figsize=(10, 8))
sb.barplot(data=actors_more_than_once, x='Count', y='japanCastMember', hue='japanCastMember', palette='viridis', ax=ax)
ax.set_title('Japanese Cast Members with More Than One Movie')
ax.set_xlabel('Number of Movies')
ax.set_ylabel('Japanese Cast Member')

# Annotate the bars with movie names
for i in range(len(actors_more_than_once)):
    ax.text(actors_more_than_once['Count'].iloc[i], i, actors_more_than_once['MovieName'].iloc[i], color='w', ha="right", va="center", fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
us_actor_movie_count = ch.groupby('USA_CastMember').size().reset_index(name='Count')
actors_more_than_once = us_actor_movie_count[us_actor_movie_count['Count'] > 1]

actors_more_than_once_movies = ch[ch['USA_CastMember'].isin(actors_more_than_once['USA_CastMember'])]
actors_movie_list = actors_more_than_once_movies.groupby('USA_CastMember')['MovieName'].apply(lambda x: ', '.join(x.drop_duplicates())).reset_index()
actors_more_than_once = actors_more_than_once.merge(actors_movie_list, on='USA_CastMember')


fig, ax = plt.subplots(figsize=(10, 8))
sb.barplot(data=actors_more_than_once, x='Count', y='USA_CastMember', hue='USA_CastMember', palette='magma', ax=ax)
ax.set_title('USA Cast Members with More Than One Movie')
ax.set_xlabel('Number of Movies')
ax.set_ylabel('American Cast Member')

for i in range(len(actors_more_than_once)):
    ax.text(actors_more_than_once['Count'].iloc[i], i, actors_more_than_once['MovieName'].iloc[i], color='w', ha="right", va="center", fontsize=9)

plt.tight_layout()
plt.show()
